In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import random 
import matplotlib.pyplot as plt
import argparse
import secrets
import json
import sys
import math 

In [3]:
import sys
sys.path.append('/usr0/home/naveenr/projects/patient_provider')

In [4]:
from patient.simulator import run_multi_seed
from patient.baseline_policies import *
from patient.lp_policies import *
from patient.utils import get_save_path, delete_duplicate_results, restrict_resources, one_shot_policy, MyEncoder

In [5]:
is_jupyter = 'ipykernel' in sys.modules

In [28]:
if is_jupyter: 
    seed        = 43
    num_patients = 1225
    num_providers = 700
    provider_capacity = 1
    noise = 0
    fairness_constraint = -1
    num_trials = 25
    utility_function = "semi_synthetic_comorbidity"
    order="uniform"
    online_arrival = False   
    new_provider = False 
    out_folder = "dynamic"
    average_distance = 20.2
    max_shown = 25
    online_scale = 1
    num_samples = 10
    verbose = True 
else:
    parser = argparse.ArgumentParser()
    parser.add_argument('--seed', help='Random Seed', type=int, default=42)
    parser.add_argument('--n_patients',         '-N', help='Number of patients', type=int, default=100)
    parser.add_argument('--n_providers',        help='Number of providers', type=int, default=100)
    parser.add_argument('--max_shown',        help='How many providers to show at most', type=int, default=25)
    parser.add_argument('--num_samples',        help='How many samples to simulate for the approx. method', type=int, default=10)
    parser.add_argument('--provider_capacity', help='Provider Capacity', type=int, default=1)
    parser.add_argument('--noise', help='Noise in theta', type=float, default=0.1)
    parser.add_argument('--average_distance', help='Maximum distance patients are willing to go', type=float, default=20.2)
    parser.add_argument('--fairness_constraint', help='Maximum difference in average utility between groups', type=float, default=-1)
    parser.add_argument('--num_trials', help='Number of trials', type=int, default=100)
    parser.add_argument('--utility_function', help='Which folder to write results to', type=str, default='uniform')
    parser.add_argument('--online_scale', help='Which folder to write results to', type=float, default=1)
    parser.add_argument('--order', help='Which folder to write results to', type=str, default='uniform')
    parser.add_argument("--online_arrival",action="store_true",help="Patients arrive one-by-one")
    parser.add_argument("--new_provider",action="store_true",help="Are we simulating a new provider matching")
    parser.add_argument("--verbose",action="store_true",help="Are we storing all the information")
    parser.add_argument('--out_folder', help='Which folder to write results to', type=str, default='policy_comparison')

    args = parser.parse_args()

    seed = args.seed
    num_patients = args.n_patients
    num_providers = args.n_providers 
    num_trials = args.num_trials
    noise = args.noise
    average_distance = args.average_distance
    fairness_constraint = args.fairness_constraint
    provider_capacity = args.provider_capacity
    max_shown = args.max_shown
    utility_function = args.utility_function
    order = args.order
    online_arrival = args.online_arrival
    new_provider = args.new_provider 
    online_scale = args.online_scale
    out_folder = args.out_folder
    verbose = args.verbose 
    
assert not(online_arrival and new_provider)
save_name = secrets.token_hex(4)  

In [29]:
results = {}
results['parameters'] = {'seed'      : seed,
        'num_patients'    : num_patients,
        'num_providers': num_providers, 
        'provider_capacity'    : provider_capacity,
        'max_shown': max_shown,
        'utility_function': utility_function, 
        'order': order, 
        'num_trials': num_trials, 
        'noise': noise, 
        'average_distance': average_distance,
        'online_arrival': online_arrival,
        'new_provider': new_provider,
        'fairness_constraint': fairness_constraint,
        'online_scale': online_scale, 
        'verbose': verbose} 

## Baselines

In [30]:
seed_list = [seed]
restrict_resources()

In [1]:
if not online_arrival and fairness_constraint == -1:
    policy = one_shot_policy 
    per_epoch_function = offer_everything
    name = "offer_everything"
    print("{} policy".format(name))

    rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

    for key in rewards:
        results['{}_{}'.format(name,key)] = rewards[key]
    print("Matches {}, Utilities {}".format(np.mean(results['offer_everything_num_matches'])/num_patients,np.mean(results['offer_everything_patient_utilities'])))

NameError: name 'online_arrival' is not defined

In [80]:
if not online_arrival and fairness_constraint == -1:
    policy = one_shot_policy
    per_epoch_function = greedy_policy
    name = "greedy"
    print("{} policy".format(name))

    rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

    for key in rewards:
        results['{}_{}'.format(name,key)] = rewards[key]
    print("Matches {}, Utilities {}".format(np.mean(results['{}_num_matches'.format(name)])/num_patients,np.mean(results['{}_patient_utilities'.format(name)])))

greedy policy


Matches 0.4699428571428571, Utilities 0.5545991262003672


In [11]:
results.keys()

dict_keys(['parameters', 'offer_everything_patient_utilities', 'offer_everything_chosen_providers', 'offer_everything_num_matches', 'offer_everything_assortments', 'offer_everything_patient_orders', 'offer_everything_new_providers', 'offer_everything_options', 'offer_everything_weights', 'greedy_patient_utilities', 'greedy_chosen_providers', 'greedy_num_matches', 'greedy_assortments', 'greedy_patient_orders', 'greedy_new_providers', 'greedy_options', 'greedy_weights'])

In [20]:
if not online_arrival and fairness_constraint == -1:
    policy = one_shot_policy
    per_epoch_function = optimal_policy
    name = "omniscient_optimal"
    print("{} policy".format(name))

    rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function,use_real=True)

    for key in rewards:
        results['{}_{}'.format(name,key)] = rewards[key]
    print("Matches {}, Utilities {}".format(np.mean(results['{}_num_matches'.format(name)])/num_patients,np.mean(results['{}_patient_utilities'.format(name)])))

omniscient_optimal policy


KeyboardInterrupt: 

## Optimization-Based

In [49]:
if not online_arrival:
    policy = one_shot_policy
    if fairness_constraint != -1:
        per_epoch_function = lambda p: lp_policy(p,fairness_constraint=fairness_constraint)
    else:
        per_epoch_function = lp_policy
    name = "lp"
    print("{} policy".format(name))

    rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

    for key in rewards:
        results['{}_{}'.format(name,key)] = rewards[key]
    print("Matches {}, Utilities {}".format(np.mean(results['{}_num_matches'.format(name)])/num_patients,np.mean(results['{}_patient_utilities'.format(name)])))

lp policy
Matches 0.0, Utilities 0.6215182405594668


In [13]:
# policy = one_shot_policy
# per_epoch_function = full_milp_policy
# name = "full_milp_policy"
# print("{} policy".format(name))

# rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

# for key in rewards:
#     results['{}_{}'.format(name,key)] = rewards[key]
# print("Matches {}, Utilities {}".format(np.mean(results['{}_num_matches'.format(name)])/num_patients,np.mean(results['{}_patient_utilities'.format(name)])))

In [54]:
# policy = one_shot_policy
# per_epoch_function = full_lp_policy
# name = "full_lp_policy"
# print("{} policy".format(name))

# rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

# for key in rewards:
#     results['{}_{}'.format(name,key)] = rewards[key]
# print("Matches {}, Utilities {}".format(np.mean(results['{}_num_matches'.format(name)])/num_patients,np.mean(results['{}_patient_utilities'.format(name)])))

In [31]:
if fairness_constraint == -1:
    policy = one_shot_policy
    per_epoch_function = lambda m: greedy_justified(m,K=10)
    name = "greedy_justified"
    print("{} policy".format(name))

    rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

    for key in rewards:
        results['{}_{}'.format(name,key)] = rewards[key]
    print("Matches {}, Utilities {}".format(np.mean(results['{}_num_matches'.format(name)])/num_patients,np.mean(results['{}_patient_utilities'.format(name)])))

greedy_justified policy


In [ ]:
if fairness_constraint == -1:
    policy = one_shot_policy
    per_epoch_function = optimal_dual_assignment_with_exit
    name = "greedy_justified_new"
    print("{} policy".format(name))

    rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

    for key in rewards:
        results['{}_{}'.format(name,key)] = rewards[key]
    print("Matches {}, Utilities {}".format(np.mean(results['{}_num_matches'.format(name)])/num_patients,np.mean(results['{}_patient_utilities'.format(name)])))

greedy_justified_new policy


[700]
Matches 0.5686204081632653, Utilities 0.6164054915505635


In [37]:
np.array(results['greedy_justified_new_assortments']).shape

(1, 100, 1)

In [12]:
np.array(results['greedy_justified_new_weights'])[0][]

(1, 25, 100, 51)

In [27]:
diffs = np.array(rewards['weights'])[0,:,:,0]-np.array(rewards['weights'])[0,:,:,-1]

In [28]:
diffs.shape

(25, 100)

In [78]:
np.sum(results['greedy_justified_assortments'][0],axis=1)

array([ 0, 12,  6, 17,  1,  5,  6, 14,  5,  2, 17, 10,  3, 18,  1, 13,  8,
        0,  4, 15,  5,  9,  0,  9,  0,  5,  2, 10,  5, 13, 16, 12,  0, 14,
        6, 18,  7, 15,  8, 18,  5, 15,  0, 12,  7,  0,  9, 15,  1, 19,  0,
        6, 14, 21,  7, 10,  3, 12,  9, 14, 15,  9, 12, 17, 15,  2, 10,  2,
       18,  3, 12,  9, 13,  0,  4, 17, 19, 12,  0,  8,  5,  0, 18,  9,  9,
        0,  9,  9,  6, 13,  0, 14,  0, 11, 12, 14,  8, 21, 12, 12])

In [77]:
np.sum(results['greedy_justified_new_assortments'][0],axis=1)

array([20, 18, 16, 16, 22, 13, 16, 16, 11, 16, 20, 14,  9, 21, 14, 18, 19,
       15,  5, 16,  8, 12, 18, 11, 10, 20, 20, 15,  9, 17, 18, 19, 18, 17,
       15, 17, 15, 18, 14, 20, 15, 16, 12, 18, 19, 18, 12, 16, 16, 20,  0,
       22, 15, 20, 16, 16, 11, 13, 13, 16, 20, 13, 16, 21, 21, 13,  7, 18,
       18,  3, 19, 16, 14,  7, 13, 19, 19, 16, 17, 12, 15, 12, 21,  4, 21,
       13, 15, 15, 13, 12, 17, 18, 11, 19, 13, 20, 12, 20, 14, 21])

In [58]:
np.mean(results['greedy_justified_new_assortments'])

np.float64(0.67)

## Save Data

In [ ]:
save_path = get_save_path(out_folder,save_name)

In [ ]:
delete_duplicate_results(out_folder,"",results)

In [ ]:
json.dump(results,open('../../results/'+save_path,'w'),cls=MyEncoder)